In [39]:
#importation des bibliothèques

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from math import sqrt
from sklearn.ensemble import RandomForestRegressor

In [40]:
#lecture des tableaux

df = pd.read_csv('tableau_pour_apprentissage.csv', sep=';')
df2 = pd.read_csv('tableau_a_remplir.csv', sep=';')

#on garde que les lignes du tableau pour apprentissage contenant les CHE
df_masque = df[df['cespece'] == 'CHE']

#on fait en sorte d'avoir autant de lignes dans le tableau pour apprentissage que dans le tableau à remplir
df_sample = df_masque.sample(n = 468)

#on garde que les données quantitatives du tableau d'apprentissage
df_quanti = df_sample[['Secteur','NumStation','Month','Year']]

In [41]:
#centre-réduction des données

scaler = StandardScaler() 
scaler.fit(df_quanti) 
df_scaled = pd.DataFrame(scaler.transform(df_quanti), columns=['Secteur','NumStation','Month','Year'])

In [42]:
#adaptation des données

df_scaled_nomiss = df_scaled.dropna()
df_features = df_scaled_nomiss.iloc[:,:-1]
df_label = df_sample['taille (cm)']

In [43]:
#séparation des données

i = 42
X_train, X_test, y_train, y_test = train_test_split(df_features, df_label, test_size=0.8, random_state=i)

In [44]:
#validation croisée via RMSE

#entraînement du modèle
model = RandomForestRegressor()
model.fit(X_train, y_train)

#prédictions
pred_train_y = model.predict(X_train)
pred_test_y = model.predict(X_test)

#évaluation du modèle
r2_train = r2_score(y_train, pred_train_y)    
r2_test = r2_score(y_test, pred_test_y)

train_MSE_norm = mean_squared_error(y_train, pred_train_y)
test_MSE_norm = mean_squared_error(y_test, pred_test_y)

train_RMSE_norm = np.sqrt(np.absolute(train_MSE_norm))
test_RMSE_norm = np.sqrt(np.absolute(test_MSE_norm))

print(r2_train, r2_test, train_MSE_norm, test_MSE_norm, train_RMSE_norm, test_RMSE_norm)

0.2035881559371835 -0.10970696546499026 68.21480060301359 72.00383656213499 8.259225181759701 8.485507442818903


In [46]:
#remplissage du dataframe

df2['taille (cm)'] = model.predict(X_test)
df2.to_csv("fichier_finalRMSE.csv", index=False)